# Building code for NYSM +/-15 day averages
- https://docs.xarray.dev/en/stable/examples/monthly-means.html
- https://docs.xarray.dev/en/stable/generated/xarray.Dataset.rolling.html#xarray.Dataset.rolling

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read in NYSM data
# Reading in one month takes 1-2 seconds, so 6 years ~1-2 min
nysm_sites = pd.read_csv("/spare11/atm533/data/nysm_sites.csv")
nysm_data = pd.read_csv("/spare11/atm533/data/nysm_data_202008.csv") 


In [3]:
# 126 sites * 12 per hour (every 5 min) * 24 hours * 31 days
# len(nysm_data)

In [4]:

# set station and time to be indices of the df so that the conversion to xarray dataset makes them coordinates
nysm_data = nysm_data.set_index(["station", "time"])

# convert to dataset
nysm_ds = nysm_data.to_xarray()

# convert time coordinate of the dataset to datetime format
# NOTE: must be executed twice (last subcomment here by Vinod: https://stackoverflow.com/questions/62572678/xarray-coords-conversion-to-datetime64)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)

nysm_ds

<xarray.Dataset>
Dimensions:                                (station: 126, time: 8929)
Coordinates:
  * station                                (station) object 'ADDI' ... 'YORK'
  * time                                   (time) datetime64[ns] 2020-07-31T2...
Data variables: (12/28)
    temp_2m [degC]                         (station, time) float64 23.3 ... 22.6
    temp_9m [degC]                         (station, time) float64 23.5 ... 22.8
    relative_humidity [percent]            (station, time) float64 57.1 ... 64.1
    precip_incremental [mm]                (station, time) float64 0.0 ... 0.0
    precip_local [mm]                      (station, time) float64 0.0 ... 0.0
    precip_max_intensity [mm/min]          (station, time) float64 0.0 ... 0.0
    ...                                     ...
    soil_temp_05cm [degC]                  (station, time) float64 22.1 ... 21.1
    soil_temp_25cm [degC]                  (station, time) float64 20.1 ... 20.3
    soil_temp_50cm [degC]                  (station, time) float64 19.1 ... 20.1
    soil_moisture_05cm [m^3/m^3]           (station, time) float64 0.08 ... 0.18
    soil_moisture_25cm [m^3/m^3]           (station, time) float64 0.2 ... 0.24
    soil_moisture_50cm [m^3/m^3]           (station, time) float64 0.27 ... 0.21

In [5]:
# Calculate the mean of each variable by averaging over station and date 
# The groupby call groups data by date, then within each date, average over the time coordinate
ds_avg_by_site_date = nysm_ds.groupby("time.date").mean("time")

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in true_divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


In [6]:
ds_avg_by_site_date

<xarray.Dataset>
Dimensions:                                (station: 126, date: 32)
Coordinates:
  * station                                (station) object 'ADDI' ... 'YORK'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 23.3 ... 1...
    temp_9m [degC]                         (station, date) float64 23.5 ... 1...
    relative_humidity [percent]            (station, date) float64 57.1 ... 80.6
    precip_incremental [mm]                (station, date) float64 0.0 ... 0.0
    precip_local [mm]                      (station, date) float64 0.0 ... 0....
    precip_max_intensity [mm/min]          (station, date) float64 0.0 ... 0.0
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 22.1 ... 1...
    soil_temp_25cm [degC]                  (station, date) float64 20.1 ... 2...
    soil_temp_50cm [degC]                  (station, date) float64 19.1 ... 2...
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 0.08 ... 0...
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 0.2 ... 0....
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 0.27 ... 0.21

In [7]:
# By date, calculate rolling average of past +/- 5 days. Adjust this to +/- 15 once I get all data
# Rolling window is set to be 5 dates (dates are days)
# Note that there is a parameter, min_periods, whose default is set to None which equates to it being equal the size of the window. In other words, the if the rolling window is 5 and centered, that means we need 2 before, term in middle, 2 terms after, so we would have NANs for the first two terms
# Center = True, as it suggests, means that at a date coordinate (a date row) it is at the center of the rolling average
# Can confirm this is working as we think by changing it to center = True and seeing that the first 2 are nans
rolling_5day_avg = ds_avg_by_site_date.rolling({"date":5}, center = True).mean()
rolling_5day_avg

<xarray.Dataset>
Dimensions:                                (station: 126, date: 32)
Coordinates:
  * station                                (station) object 'ADDI' ... 'YORK'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 nan ... nan
    temp_9m [degC]                         (station, date) float64 nan ... nan
    relative_humidity [percent]            (station, date) float64 nan ... nan
    precip_incremental [mm]                (station, date) float64 nan ... nan
    precip_local [mm]                      (station, date) float64 nan ... nan
    precip_max_intensity [mm/min]          (station, date) float64 nan ... nan
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_25cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_50cm [degC]                  (station, date) float64 nan ... nan
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 nan ... nan

## REFERENCE CODE ONLY
#### Ignore..


In [109]:
# call on a coordinate
nysm_ds.station

# call on a varibale
nysm_ds['temp_2m [degC]']

<xarray.DataArray 'temp_2m [degC]' (station: 126, time: 8929)>
array([[23.3, 22.7, 22.5, ..., 18.2, 18.2, 18.2],
       [21.3, 20.4, 19.7, ..., 16.8, 16.5, 16.4],
       [24.1, 24. , 24. , ..., 22.6, 22.6, 22.5],
       ...,
       [24.7, 24.2, 23.5, ..., 14.3, 14.3, 14.4],
       [23.8, 22.9, 22.5, ..., 22.1, 21.8, 21.8],
       [22.8, 22.2, 21.8, ..., 22.9, 22.8, 22.6]])
Coordinates:
  * station  (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * time     (time) datetime64[ns] 2020-07-31T23:50:00 ... 2020-08-31T23:55:00

In [ ]:
nysm_ds['temp_2m [degC]'].shape

In [29]:
# REFERENCE ONLY
# "data array" . mean ("dimension to average across"... eliminating that dim completely, it's averaged out if left blank, will average over everything to give ONE mean value)
# produces another array whose length depends on what you averaged over!

# example 1
avg_temp_by_day = nysm_ds['temp_2m [degC]'].mean('station')
print(avg_temp_by_day.shape)

# example 2
avg_temp_by_station = nysm_ds['temp_2m [degC]'].mean('time')
print(avg_temp_by_station.shape)

(8929,)
(126,)


In [39]:
# this is just like SQL - using all data, but group certain rows together based on common (i.e. season) and then still want to average over the time dimension so that dim is gone and we're left with station only
test1 = nysm_ds.groupby("time.season").mean("time")

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in true_divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


In [43]:
test1

<xarray.Dataset>
Dimensions:                                (station: 126, date: 32)
Coordinates:
  * station                                (station) object 'ADDI' ... 'YORK'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 23.3 ... 1...
    temp_9m [degC]                         (station, date) float64 23.5 ... 1...
    relative_humidity [percent]            (station, date) float64 57.1 ... 80.6
    precip_incremental [mm]                (station, date) float64 0.0 ... 0.0
    precip_local [mm]                      (station, date) float64 0.0 ... 0....
    precip_max_intensity [mm/min]          (station, date) float64 0.0 ... 0.0
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 22.1 ... 1...
    soil_temp_25cm [degC]                  (station, date) float64 20.1 ... 2...
    soil_temp_50cm [degC]                  (station, date) float64 19.1 ... 2...
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 0.08 ... 0...
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 0.2 ... 0....
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 0.27 ... 0.21

In [38]:
test2= nysm_ds.groupby("time.season")
type(test2)

xarray.core.groupby.DatasetGroupBy

In [10]:
ds_sub = nysm_ds.sel(time=nysm_ds.time.dt.month.isin([4]))

In [13]:
nysm_ds.time.dt.date

<xarray.DataArray 'date' (time: 8929)>
array([datetime.date(2020, 7, 31), datetime.date(2020, 8, 1),
       datetime.date(2020, 8, 1), ..., datetime.date(2020, 8, 31),
       datetime.date(2020, 8, 31), datetime.date(2020, 8, 31)],
      dtype=object)
Coordinates:
  * time     (time) datetime64[ns] 2020-07-31T23:50:00 ... 2020-08-31T23:55:00

In [99]:
# REFERENCE -- Check that this rolling average is working correctly, not getting mixed up by sites or anything

# SUBSET DATASET: https://stackoverflow.com/questions/38846323/python-xarray-dataset-sel-select-multiple-values-along-one-dimension
# way 1
y = ds_avg_by_site_date.where(ds_avg_by_site_date.station=='WOLC', drop = True)#, "date" :'2020-08-06'})
y

# way 2 (using sel and also datetime issue)
# Using .sel (datetime format issue..)
# g = ds_avg_by_site_date.sel(ds_avg_by_site_date.date == dt.date(2020, 7, 31)) # DATETIME ISSUE
# g

# print out the temps for WOLC then avg in cell below to make sure it works
y["temp_2m [degC]"]

<xarray.DataArray 'temp_2m [degC]' (station: 1, date: 32)>
array([[23.8       , 22.29583333, 25.55173611, 23.64409722, 20.59097222,
        20.34097222, 18.16458333, 19.39201389, 21.21284722, 22.58715278,
        25.67291667, 26.40486111, 22.86041667, 21.67986111, 22.54895833,
        22.99444444, 22.41805556, 20.709375  , 19.25034722, 18.06284722,
        18.01909722, 20.62152778, 22.36423611, 23.75208333, 24.45972222,
        23.28298611, 18.26666667, 20.33854167, 20.16666667, 21.65798611,
        18.71840278, 17.40381944]])
Coordinates:
  * station  (station) object 'WOLC'
  * date     (date) object 2020-07-31 2020-08-01 ... 2020-08-30 2020-08-31

In [8]:
num = 23.8 +22.29583333+25.55173611+23.64409722+20.59097222
num/5

23.176527776

In [9]:
# check that this was the first non nan value for WOLC
# rolling_5day_avg
rolling_5day_avg.where(rolling_5day_avg.station=='WOLC', drop = True) # confirmed! the first value is 23.18

<xarray.Dataset>
Dimensions:                                (station: 1, date: 32)
Coordinates:
  * station                                (station) object 'WOLC'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 nan ... nan
    temp_9m [degC]                         (station, date) float64 nan ... nan
    relative_humidity [percent]            (station, date) float64 nan ... nan
    precip_incremental [mm]                (station, date) float64 nan ... nan
    precip_local [mm]                      (station, date) float64 nan ... nan
    precip_max_intensity [mm/min]          (station, date) float64 nan ... nan
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_25cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_50cm [degC]                  (station, date) float64 nan ... nan
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 nan ... nan